In [17]:
from openai import OpenAI
import os
from PIL import Image
import base64
from io import BytesIO
api_key=os.getenv('openai_api_token')

client = OpenAI()
def convert_image_to_base64(image_path):
    with Image.open(image_path) as image:
        # Convert image to RGB
        rgb_image = image.convert('RGB')
        # Create a BytesIO object to hold the image data
        buffered = BytesIO()
        # Save the image to the BytesIO object in JPEG format
        rgb_image.save(buffered, format="JPEG")
        # Get the byte data from the BytesIO object
        image_byte_data = buffered.getvalue()
        # Encode the byte data in base64
        base64_encoded_image = base64.b64encode(image_byte_data)
        # Convert the base64 bytes to string
        return base64_encoded_image.decode()

def image_encode(image_path):
    with open(image_path, "rb") as image_file:
        #encoded_string = base64.b64encode(image_file.read())
        return base64.b64encode(image_file.read()).decode('utf-8')


image_path = "./gpt-4v-demos/test_images/soup_can.jpg"

In [18]:
response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image?"},
        {
          "type": "image_url",
          "image_url": {
            #"url": {"url": f"data:image/jpeg;base64,{image_encode(image_path)}"},
            "detail": "high",
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This image features a wooden boardwalk stretching through a lush green meadow. The meadow is dense with tall grasses, and the vegetation extends into the distance with a variety of shrubs and trees scattering the horizon. Above, the sky is painted with a soft palette of blues and a few patches of white clouds, suggesting a calm, sunny day. The boardwalk provides a pathway inviting one to explore the natural setting while protecting the ground underneath from being trampled, often used in ecologically sensitive areas to prevent erosion and disturbance to plant life.', role='assistant', function_call=None, tool_calls=None))


In [35]:
import requests


headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4-vision-preview",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What’s in this image?"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{image_base64}"
            #"url": f"data:image/jpeg;base64,{image_encode(image_path)}",
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}
response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
print(response.json())

{'id': 'chatcmpl-8cie38mOkujVcrjvR353ymApRMxCv', 'object': 'chat.completion', 'created': 1704238771, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 778, 'completion_tokens': 125, 'total_tokens': 903}, 'choices': [{'message': {'role': 'assistant', 'content': 'The image shows a can of Campbell\'s Tomato Soup. The can is designed with the familiar red and white Campbell\'s branding, with the words \'Condensed\' and \'Tomato Soup\' prominently displayed. The label also indicates that the soup has 90 calories per 1/2 cup serving. There is a gold medal on the label that says "Paris International Exposition 1900," indicating an award the company received. The net weight of the contents is listed as 10.75 OZ (305g). The Campbell\'s brand is well-recognized and often associated with an iconic piece of American pop culture.'}, 'finish_reason': 'stop', 'index': 0}]}


In [36]:
response.json()['choices'][0]['message']['content'].replace("\n", "",2)

'The image shows a can of Campbell\'s Tomato Soup. The can is designed with the familiar red and white Campbell\'s branding, with the words \'Condensed\' and \'Tomato Soup\' prominently displayed. The label also indicates that the soup has 90 calories per 1/2 cup serving. There is a gold medal on the label that says "Paris International Exposition 1900," indicating an award the company received. The net weight of the contents is listed as 10.75 OZ (305g). The Campbell\'s brand is well-recognized and often associated with an iconic piece of American pop culture.'

In [33]:
from io import BytesIO

def convert_image_to_base64(image_path):
    # Open the image
    with Image.open(image_path) as image:
        # Convert the image to RGB (optional, depends on your requirements)
        rgb_image = image.convert('RGB')
        # Prepare the buffer
        buffered = BytesIO()
        # Save the image to the buffer
        rgb_image.save(buffered, format="JPEG")
        # Get the byte data
        img_data = buffered.getvalue()
        # Encode to base64
        base64_encoded = base64.b64encode(img_data)
        return base64_encoded.decode()

In [34]:
image_base64 = convert_image_to_base64(image_path)
print(image_base64)

...
